In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import keras
from keras.models import Sequential
from keras.layers.core import Dense
from keras.layers import Bidirectional, LSTM, Activation, Dropout, Embedding, Input
from keras import regularizers
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

import pandas as pd

import numpy as np
import json
import math

import os.path

def save_log(loglist, filename): #save a list of lists 
  df = pd.DataFrame.from_records(loglist)
  df.to_csv(filename, index=False)

def remove_nan(lists):
  newlists = []
  for tr in lists:
    newlists.append([int(x) for x in tr if str(x) != 'nan'])
  return(newlists)

def import_log(filepath):
  df = pd.read_csv(filepath)
  return(remove_nan(df.values.tolist()))



def number_to_one_hot_X(X, dict_size): #if we want 
  newX = []
  for example in X:
    new_ex = []
    for i in range(len(example)):
      onehot = [0]*dict_size #changed
      if example[i] != 0:
        onehot[example[i] - 1] = 1 #-1 because begin counting at 0
      new_ex.append(onehot)
    newX.append(new_ex)
  return(np.array(newX))

def create_XY_prefix(log, mappingsize, prefixlen):
  X = []
  Y = []
  for i in range(0, len(log)):
    for k in range(1, len(log[i])):
      X.append(log[i][max(0, k-prefixlen):k]) #get the prefix of 'encoded' activities
      y = [0] *(mappingsize)
      y[int(log[i][k])-1] = 1
      Y.append(y)        
  X = keras.preprocessing.sequence.pad_sequences(X, maxlen=prefixlen, padding='pre')
  X = number_to_one_hot_X(X, mappingsize)
  return(np.array(X), np.array(Y))

def get_startend(log): 
  return log[0][0], log[0][-1]

def get_model(maxlen, num_chars, bidirec, n_layers, lstmsize, dropout, l1, l2):
  model = Sequential()
  model.add(Input(shape=(maxlen, num_chars))) #If you don't use an embedding layer input should be one-hot-encoded
  if bidirec == False:   
    model.add(LSTM(lstmsize,kernel_initializer='glorot_uniform',return_sequences=(n_layers != 1),kernel_regularizer=regularizers.l1_l2(l1,l2),
                   recurrent_regularizer=regularizers.l1_l2(l1,l2),input_shape=(maxlen, num_chars)))
    model.add(Dropout(dropout))
    for i in range(1, n_layers):
      return_sequences = (i+1 != n_layers)
      model.add(LSTM(lstmsize,kernel_initializer='glorot_uniform',return_sequences=return_sequences,
                     kernel_regularizer=regularizers.l1_l2(l1,l2),recurrent_regularizer=regularizers.l1_l2(l1,l2)))
      model.add(Dropout(dropout))
  else:
    model.add(Bidirectional(LSTM(lstmsize,kernel_initializer='glorot_uniform',return_sequences=(n_layers != 1),kernel_regularizer=regularizers.l1_l2(l1,l2),
                   recurrent_regularizer=regularizers.l1_l2(l1,l2),input_shape=(maxlen, num_chars))))
    model.add(Dropout(dropout))
    for i in range(1, n_layers):
      return_sequences = (i+1 != n_layers)
      model.add(Bidirectional(LSTM(lstmsize,kernel_initializer='glorot_uniform',return_sequences=return_sequences,
                     kernel_regularizer=regularizers.l1_l2(l1,l2),recurrent_regularizer=regularizers.l1_l2(l1,l2))))
      model.add(Dropout(dropout))
  model.add(Dense(num_chars, kernel_initializer='glorot_uniform',activation='softmax'))
  opt = Adam(learning_rate=0.005)
  model.compile(loss='categorical_crossentropy', optimizer=opt, metrics='accuracy')
  return model


def train_model(X_train, y_train,batch_size, maxlen, num_chars, bidirec, n_layers, lstmsize, dropout, l1, l2):
  model = get_model(maxlen, num_chars, bidirec, n_layers, lstmsize, dropout, l1, l2)
  model.summary()
  early_stopping = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)
  lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=0, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)
  #train_model
  history = model.fit(X_train, y_train, validation_split=0.2, callbacks=[early_stopping, lr_reducer], batch_size=batch_size, epochs=600, verbose=2)
  return model

def cut_end(log, endact):
  logsize, tracesize = log.shape
  print(log.shape)
  newlog = []
  for i in range(0, logsize):
    trace = []
    for j in range(0, tracesize):
      if log[i][j] == endact:
        trace.append(log[i][j])
        break
      else:
        trace.append(log[i][j])
    newlog.append(trace)
  return(newlog)

def normalize(probs): #normalize probabilities to sum to 1
  examplesize, actsize = probs.shape
  newy = []
  for i in range(examplesize):
    normalizer = 1 / float( sum(probs[i]) )
    ynorm = [float(l) * normalizer for l in probs[i]]
    newy.append(ynorm)
  return newy


def choose_act_all(all_y): #randomly choose an activity, stochastically
  #p want a list of probabilities    
  chosen_acts = []
  for i in range(len(all_y)):
      chosen_acts.append(np.random.choice(np.arange(0, len(all_y[i])), p=all_y[i])+1)  
  return(chosen_acts)   # +1 because number encodig starts at 1 not 0

def OHget_probabilities(rnnmodel, xlists,  nr_act, maxlen, prefixlen):
  #assume xlist is a list with the x (prefix) untill now 
  all_x = keras.preprocessing.sequence.pad_sequences(xlists, maxlen=maxlen, padding="pre")
  all_x = all_x[:,-(prefixlen):]
  all_x = number_to_one_hot_X(all_x, nr_act)
  results = rnnmodel.predict(all_x)
  return results

def OHsimulate_log(RNNmodel, logsize, startact, endact, maxlen, mapping, prefixlen): #Use RNN to simulate log
  log = np.zeros((logsize, maxlen+1), int)
  for i in range(0, logsize): #start every trace with the start activity
    log[i][0] = startact
  print(log)
  for j in range(1,maxlen+1): #check if 0 or 1 and ml or ml - 1 #we took 50 for with loops   
    print("finding activity nr", j+1)   
    prefixes = np.array([log[i][0:j] for i in range(0, logsize)])
    print(prefixes)
    probs = OHget_probabilities(RNNmodel, prefixes, len(mapping), maxlen, prefixlen)
    #we need to do this because otherwise probabilities sum over 1 
    ynorm = normalize(probs) 
    nextacts = choose_act_all(ynorm) 
    for i in range(0, logsize):
      log[i][j] = nextacts[i]
  print(log)
  corrected_log = cut_end(log, endact)      
  return(corrected_log) 


def do_grid_search_without_embedding_one_variant(variant, prefixlen):
  trainname = '/content/drive/MyDrive/Grid/Training_logs/log_'+ str(variant) + '.csv'
  #reload mapping
  mappingfilename = '/content/drive/MyDrive/Grid/mapping.txt' 
  with open(mappingfilename) as f:
    mapping = json.loads(f.read())

  train_log = remove_nan(import_log(trainname))
  X_train, y_train = create_XY_prefix(train_log, len(mapping), prefixlen)

  maxlen = len(max(train_log,key=len))

  bidirec = [True, False]
  grid_nr_layers =  [1, 2]
  grid_layersize = [16, 32, 64]
  grid_dropout = [0.0, 0.2, 0.4, 0.6]
  grid_reg = [0.0, 0.00001, 0.0001, 0.001, 0.01]
  for bi in bidirec:
    for num_layers in grid_nr_layers:
      for layersize in grid_layersize:
        for dropout in grid_dropout:
          for reg in grid_reg:
            if bi == False:
              filename = '/content/drive/MyDrive/Grid/Simulated_logs/Var'+str(variant)+'/SIMLOG_NL'+str(num_layers)+'emb'+'N'+'LS'+str(layersize)+'D'+str(dropout).replace('0.', '')+'R'+str(reg).replace('.', '')+'.csv'
            else:
              filename = '/content/drive/MyDrive/Grid/Simulated_logs/Var'+str(variant)+'/SIMLOG_BIDIREC_NL'+str(num_layers)+'emb'+'N'+'LS'+str(layersize)+'D'+str(dropout).replace('0.', '')+'R'+str(reg).replace('.', '')+'.csv'
            if os.path.exists(filename): 
              print("Already exists:", num_layers, layersize, dropout, reg)
              continue

            batch_size = 128
            model = train_model(X_train, y_train,batch_size, maxlen=prefixlen, num_chars=len(mapping), bidirec=bi, n_layers=num_layers, lstmsize=layersize, dropout=dropout, l1=reg, l2=reg)
            model.summary()
           
            #save RNN model in case we ever need it
            #modelfilename = '/content/drive/MyDrive/DataExperiment/Models/Var'+str(variant)+'/model_NL'+str(num_layers)+'emb'+'N'+'LS'+str(layersize)+'D'+str(dropout).replace('0.', '')+'R'+str(reg).replace('.', '')
            #model.save(modelfilename)

            start,end = get_startend(train_log)

            simlog = OHsimulate_log(model, 12000, start, end, maxlen-1, mapping, prefixlen)

            #save simulated log
            save_log(simlog, filename)

Mounted at /content/drive


In [ ]:
do_grid_search_without_embedding_one_variant(3, 10)

Streaming output truncated to the last 5000 lines.
finding activity nr 11
[[ 6  7  8 ...  4  1 10]
 [ 6  7  8 ...  4 11 10]
 [ 6  7 13 ...  2  2 11]
 ...
 [ 6  7  8 ...  1  1 10]
 [ 6  7  8 ...  2  4 10]
 [ 6  7  8 ...  4  3 10]]
finding activity nr 12
[[ 6  7  8 ...  1 10  5]
 [ 6  7  8 ... 11 10  5]
 [ 6  7 13 ...  2 11 10]
 ...
 [ 6  7  8 ...  1 10  5]
 [ 6  7  8 ...  4 10  5]
 [ 6  7  8 ...  3 10  5]]
finding activity nr 13
[[ 6  7  8 ... 10  5 12]
 [ 6  7  8 ... 10  5 12]
 [ 6  7 13 ... 11 10 11]
 ...
 [ 6  7  8 ... 10  5 12]
 [ 6  7  8 ... 10  5 12]
 [ 6  7  8 ... 10  5 12]]
[[ 6  7  8 ...  5 12  9]
 [ 6  7  8 ...  5 12  9]
 [ 6  7 13 ... 10 11  7]
 ...
 [ 6  7  8 ...  5 12  9]
 [ 6  7  8 ...  5 12  9]
 [ 6  7  8 ...  5 12  9]]
(12000, 13)
Already exists: 1 64 0.0 0.0
Already exists: 1 64 0.0 1e-05
Already exists: 1 64 0.0 0.0001
Already exists: 1 64 0.0 0.001
Already exists: 1 64 0.0 0.01
Already exists: 1 64 0.2 0.0
Already exists: 1 64 0.2 1e-05
Already exists: 1 64 0.2 0.0001